# Web Scraping Homework - Mission to Mars

In [1]:
#import dependencies
import os
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup config variables to enable Splinter interaction with browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\jchan\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


<strong> Hint:</strong> Use Splinter to navigate the sites when needed and BeautifulSoup to help find and parse out the necessary data.

In [3]:
# Create dictionary to store news
nasa_data = {}

## NASA Mars News

Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [4]:
# Visit Nasa news url through splinter module
nasa_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(nasa_url)

In [5]:
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [6]:

# Get news title and news text by searching for appropriate div class 
news_title = soup.find('li', class_='slide').find('div', class_='content_title').text
news_p = soup.find('li', class_='slide').find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

NASA's Perseverance Rover Collects Puzzle Pieces of Mars' History
The rocks it has analyzed for sample collection are helping the team better understand a past marked by volcanic activity and water.


In [7]:
# Create dictionary to store data and save entries
scrape_nasa_news={"Title":news_title, "Paragraph":news_p}
scrape_nasa_news

{'Title': "NASA's Perseverance Rover Collects Puzzle Pieces of Mars' History",
 'Paragraph': 'The rocks it has analyzed for sample collection are helping the team better understand a past marked by volcanic activity and water.'}

In [8]:
# Save scraped data as a new entry in the dictionary
nasa_data ["Title"] = news_title
nasa_data["Paragraph"] = news_p

## JPL Mars Space Images - Featured Image

- Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).
- Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
- Find the image url to the full size .jpg image. Make sure to save a complete url string for this image.

In [13]:
# JPL URL
jpl_url_img = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url_img)

In [14]:
image_html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(image_html, 'html.parser')

In [15]:
# Get featured image
feature_img_url_path = soup.find("article", class_ ="main_img")["src"]

mars_feature_img = "https://www.jpl.nasa.gov/" + feature_img_url_path

mars_feature_img

TypeError: 'NoneType' object is not subscriptable

In [16]:
feature_img_url

NameError: name 'feature_img_url' is not defined

## Mars Facts

Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com/) and use Pandas to scrape the table containing facts about the planet.
Use Pandas to convert the data to a HTML table string.

In [18]:
facts_url = "https://galaxyfacts-mars.com/"
facts_data = pd.read_html(facts_url)

facts_df = facts_data[0]
facts_table = facts_df.to_html(index=False)
facts_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>0</th>\n      <th>1</th>\n      <th>2</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Mars - Earth Comparison</td>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <td>Temperature:</td>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'